# LaBr3 波形解堆积算法

### 数据

- 每轮约 20 万个长波形，每个波形 20000 个点

### 触发逻辑

- 快速滤波 (fastfilter) 过阈触发
   - 前 1000 个点无信号，用于评估噪声水平
   - 取噪声水平的若干倍为阈值
   
### 标准波形与峰宽修正

- 取不存在堆积的波形进行分析
   - 堆积即触发前后一定范围存在其他触发
- 计算波形半高宽，画出峰宽-峰高的二维关联图
- 在该二维图上用 TCutG 选取密集条带区域
   - 条带外的散点来自非常紧密的堆积事件，它们无法用快速滤波分辨出来
- 用二次函数在 TCutG 范围内拟合峰宽-峰高关系
   - 拟合时将统计在峰高维度上进行平均化处理，即随机丢弃一些事件，使二维图在峰高轴上的投影谱贴近一常数
- 在 TCutG 范围内，分别计算标准中子波形和标准gamma波形
   - 标准波形按照高度进行归一
   - 根据峰宽-峰高关系，通过时间轴缩放的方式将峰宽归一到一特定值

### 波形解堆积与拟合

- 将原始长波形分段，堆积的信号一起拟合，无堆积的信号相互分离
- 以触发位置为初值，进行第一次拟合
   - 拟合时，将峰宽与峰高绑定，即根据峰高计算峰宽，然后对波形在时间轴上进行缩放
- 计算拟合结果与原波形的残差，对残差图寻峰，在峰值处尝试添加新峰拟合
   - 添加新峰，要求残差大于噪声水平，且新峰拟合后的位置不能与已有峰重合，且必须将 chi2/ndf 降低到一定比例以下
   - 若以上条件不满足，则选取残差的另一峰值尝试
   - 若确定添加新峰，则再次计算拟合残差，重复本步骤
   - 若所有残差峰值处都不能添加新峰，则结束循环
- 最后再次检查并剔除位置重合的峰（添加新峰可能会影响原有峰位）

In [1]:
!date

2023年 10月 28日 星期六 16:37:19 CST

